<div align="center">
	<h1>CS 328: Introduction to Data Science</h1>
	<h2>Assignment 2</h2>
	Shardul Junagade <br>
	23110297
</div>

### Importing Libraries

In [ ]:
# !uv pip install networkx matplotlib pandas numpy tqdm
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm

## Question 1

Q1. Implement the greedy algorithm for the densest subgraph. Run your algorithm on the [linked dataset](https://snap.stanford.edu/data/facebook-large-page-page-network.html). Report the density of the component extracted, as well as the histogram of the page categories in the densest subgraph. Also report the overall histogram of page categories and comment on how similar/different the two histograms are. Also compare its density with the density of the whole graph.


In [ ]:
# Function to compute the density of a subgraph
def compute_density(subgraph):
    if subgraph.number_of_nodes() == 0:
        return 0
    return subgraph.number_of_edges() / subgraph.number_of_nodes()


def greedy_densest_subgraph(G):
    """
    Implementation of the greedy 2-approximation algorithm for the densest subgraph problem.
    This algorithm iteratively removes the node with the minimum degree until no edges are left.
    The subgraph with the maximum density is returned.

    Parameters:
    G (networkx.Graph): The input graph.
    """
    # Create a copy of the original graph to work on
    current_subgraph = G.copy()
    if current_subgraph.number_of_nodes() == 0:
        return None, 0, [], []
    
    best_density = compute_density(current_subgraph)
    best_subgraph = current_subgraph.copy()
    densities = [best_density]
    removal_order = []

    # Use tqdm to display progress
    with tqdm(total=current_subgraph.number_of_nodes(), desc="Removing nodes") as pbar:
        while current_subgraph.number_of_nodes() > 0:
            min_degree_node = min(current_subgraph.nodes(), key=lambda v: current_subgraph.degree(v))
            removal_order.append(min_degree_node)
            current_subgraph.remove_node(min_degree_node)
            density = compute_density(current_subgraph)
            densities.append(density)
            if density > best_density:
                best_density = density
                best_subgraph = current_subgraph.copy()
            pbar.update(1)
    
    print("\nDensest subgraph statistics:")
    print(f"Number of nodes: {best_subgraph.number_of_nodes()}")
    print(f"Number of edges: {best_subgraph.number_of_edges()}")
    print(f"Density: {best_density}")
    print(f"Removal order: {removal_order}")
    
    # Create a plot of density
    plt.figure(figsize=(10, 6))
    plt.plot(densities, marker='o')
    plt.title('Density of the subgraph during removal of nodes')
    plt.xlabel('Number of nodes removed')
    plt.ylabel('Density')
    plt.grid()
    plt.savefig('density_plot.png')
    plt.close()

    return best_subgraph, best_density, removal_order, densities

In [5]:
facebook_root = "./data/facebook_large/"

node_attrs = {}
with open(facebook_root + "musae_facebook_target.csv", newline='', encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # row['id'] is the node id (as a string), row['page_type'] is the category
        node_attrs[int(row['id'])] = {
            'facebook_id': row['facebook_id'],
            'page_name': row['page_name'],
            'page_type': row['page_type']
        }

# Load the edge list
edges_df = pd.read_csv(facebook_root + "musae_facebook_edges.csv", delimiter=",", header=0)
edges = edges_df.values.tolist()

# Create a undirected NetworkX graph and add node attributes
G = nx.Graph()
G.add_edges_from(edges)
for node, attrs in node_attrs.items():
    G.nodes[node].update(attrs)

In [6]:
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())
print("Density of the whole graph:", compute_density(G))
print("Number of connected components:", nx.number_connected_components(G))
print("\nFirst 5 nodes and their attributes:")
for node in list(sorted(G.nodes(data=True)))[:5]:
    print("     ", node)


Number of nodes: 22470
Number of edges: 171002
Density of the whole graph: 7.610235870048954
Number of connected components: 1

First 5 nodes and their attributes:
      (0, {'facebook_id': '145647315578475', 'page_name': 'The Voice of China 中国好声音', 'page_type': 'tvshow'})
      (1, {'facebook_id': '191483281412', 'page_name': 'U.S. Consulate General Mumbai', 'page_type': 'government'})
      (2, {'facebook_id': '144761358898518', 'page_name': 'ESET', 'page_type': 'company'})
      (3, {'facebook_id': '568700043198473', 'page_name': 'Consulate General of Switzerland in Montreal', 'page_type': 'government'})
      (4, {'facebook_id': '1408935539376139', 'page_name': 'Mark Bailey MP - Labor for Miller', 'page_type': 'politician'})


In [ ]:
run_greedy = True
if run_greedy:
    densest_subgraph, best_density, removal_order, densities = greedy_densest_subgraph(G)
    nx.write_gml(densest_subgraph, "densest_subgraph.gml")
else:
    # load saved densest subgraph
    densest_subgraph = nx.read_gml("densest_subgraph.gml")
    best_density = compute_density(densest_subgraph)

In [ ]:
best_subgraph, best_density, removal_order, densities = greedy_densest_subgraph(G)

Removing nodes:  35%|███▍      | 7807/22470 [1:52:02<2:59:28,  1.36it/s]

In [ ]:
print(f"Density of the entire graph: {compute_density(G)}")
print(f"Density of the densest subgraph: {best_density}")

# Save the densest subgraph to a file
nx.write_gml(best_subgraph, "densest_subgraph.gml")